In [125]:
import numpy as np
import pandas as pd
import pickle
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score
from sklearn.pipeline import Pipeline

О сборе датасета: я скачала несколько репозиториев с гитхаба и пропустила их через скрипт `parser.py`. Старалась подбирать репозитории с короткими файлами, но чтобы их было много. В основном брала репозитории, в которые люди выкладывали решения задач курса по какому-то языку. Например, файлы на С++ - это конспекты курса по С++, который я проходила год назад.

Также в датасет добавлены файл, который получился на практике по данным из телеграма.

# Easy

## Предобработка

In [126]:
df = pd.read_csv('../../data/github_code.csv')
df.head()

,Unnamed: 0,text,language
0,0,import service\n\n\ndef main():\n show_head...,PYTHON
1,1,from collections import namedtuple\nfrom xml.e...,PYTHON
2,2,"def show_header():\n print(""---------------...",PYTHON
3,3,import random\nfrom shared_libs import ui_help...,PYTHON
4,4,import pandas as pd\nimport matplotlib.pyplot ...,PYTHON


In [127]:
df['language'].value_counts()

language
JAVASCRIPT    1019
PYTHON         818
CPLUSPLUS      708
Name: count, dtype: int64

In [128]:
df_tg = pd.read_csv('../../data/tg_code.csv')
df_tg['language'].value_counts()

language
text             79845
CPLUSPLUS          208
YAML                78
HASKELL             69
SQL                 69
FORTRAN             59
PYTHON              55
POWERSHELL          53
PERL                44
JAVA                41
TYPESCRIPT          40
RUST                39
R                   31
JULIA               30
PHP                 29
OCAML               28
FSHARP              27
JAVASCRIPT          26
DELPHI              25
GO                  24
COBOL               23
ASSEMBLY            22
ELIXIR              19
LATEX               18
KOTLIN              18
CRYSTAL             17
C                   17
ELM                 16
PASCAL              15
ERLANG              15
SCALA               14
AUTOHOTKEY          14
LUA                 14
FORTH               13
DART                13
CLOJURE             12
NIM                 11
BASIC               10
D                   10
RUBY                10
BATCH                9
eth                  8
1S_ENTERPRISE        8
CS

In [129]:
df_tg = df_tg.loc[df_tg['language'].isin(['CPLUSPLUS', 'PYTHON', 'JAVASCRIPT', 'text'])]
df_tg['language'].value_counts()

language
text          79845
CPLUSPLUS       208
PYTHON           55
JAVASCRIPT       26
Name: count, dtype: int64

In [130]:
df_tg['is_tg'] = True
df['is_tg'] = False
df = pd.concat([df_tg, df])
df.reset_index(drop=True, inplace=True)
filtered_df = df[df['language'] == 'text']
rows_to_remove = filtered_df.sample(n=79000, random_state=42).index
df.drop(rows_to_remove, inplace=True)
df.drop(columns=['Unnamed: 0'], inplace=True)
df.to_csv('../../data/code_merged.csv')

In [131]:
df['language'].value_counts()

language
JAVASCRIPT    1045
CPLUSPLUS      916
PYTHON         873
text           845
Name: count, dtype: int64

In [132]:
def preprocess_code(code, language):
    if language in ['CPLUSPLUS', 'PYTHON', 'JAVASCRIPT']:
        code = re.sub(r'#.*?(\n|$)', '', code)
        code = re.sub(r'//.*?(\n|$)', '', code)
        code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    
    if language != 'text':
        code = re.sub(r'[а-яА-ЯёЁ]+', '', code)
    
    code = re.sub(r'\s+', ' ', code)
    return code

In [133]:
# languages = list(df['language'])
snippets = [preprocess_code(str(df['text'][i]), str(df['language'][i])) for i in df.index]
df['text'] = snippets

## Обучение

Используем tfidf + наивный байесовский классификатор. На практике использовали tfidf для выделения фичей из пунктуации, он дал не самый лучший скор. Я предпочту использовать его же, но для всего текста, т.к. на мой взгляд некоторые синтаксические конструкции могут быть важной фичей некоторых языков программирования. В пару к нему возьмем наивный байес, т.к. он хорошо может определить, какие синтаксические конструкции в каком языке встречаются чаще.

In [134]:
train, test = train_test_split(df, test_size=0.2, random_state=42)
X_train = train['text']
X_test = test['text']
y_train = train['language']
y_test = test['language']
# X_train, X_test, y_train, y_test = train_test_split(snippets, languages, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [135]:
classifier = MultinomialNB()
classifier.fit(X_train_vect, y_train)

y_pred = classifier.predict(X_test_vect)
print(f'Balanced accuracy: {balanced_accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Balanced accuracy: 0.9113660932042744
              precision    recall  f1-score   support

   CPLUSPLUS       0.95      0.94      0.95       177
  JAVASCRIPT       0.85      0.97      0.91       205
      PYTHON       0.92      0.92      0.92       173
        text       0.95      0.81      0.88       181

    accuracy                           0.91       736
   macro avg       0.92      0.91      0.91       736
weighted avg       0.92      0.91      0.91       736



Отдельно провалидируемся на сэмплах из телеграма.

In [136]:
test_tg = test[test['is_tg'] == True]
X_test_tg = test_tg['text']
y_test_tg = test_tg['language']
X_test_tg_vect = vectorizer.transform(X_test_tg)

y_pred_tg = classifier.predict(X_test_tg_vect)
print(f'Balanced accuracy: {balanced_accuracy_score(y_test_tg, y_pred_tg)}')
print(classification_report(y_test_tg, y_pred_tg))

Balanced accuracy: 0.7930145194437772
              precision    recall  f1-score   support

   CPLUSPLUS       0.86      0.80      0.83        46
  JAVASCRIPT       0.22      1.00      0.36         9
      PYTHON       0.43      0.56      0.49        18
        text       1.00      0.81      0.90       181

    accuracy                           0.80       254
   macro avg       0.63      0.79      0.64       254
weighted avg       0.91      0.80      0.84       254



Предполагаю, что модель может путать пару javascript и python в силу схожего синтаксиса. Поскольку качество датасета сообщений из телеграма не идеально, дело может быть либо в неверном парсинге, либо в целом в неверном понимании синтаксиса моделью. При это наблюдается явный перекос в сторону javascipt (у него recall = 1) - возможно, это из-за легкого дисбаланса классов в датасете.

Сохраним модель, обученную на всем датасете, для улучшения итогового качества.

In [114]:
pipeline = Pipeline(
    [
        ("vectorizer", TfidfVectorizer()),
        ("model", MultinomialNB()),
    ]
)
pipeline.fit(df['text'], df['language'])

Pipeline(steps=[('vectorizer', TfidfVectorizer()), ('model', MultinomialNB())])

In [115]:
with open('model_prog.pickle', 'wb') as file:
    pickle.dump(pipeline, file)

## Тест скрипта

In [137]:
! python3 main.py python_code1.txt

PYTHON


In [138]:
! python3 main.py python_code2.txt

PYTHON


In [139]:
! python3 main.py python_code3.txt

PYTHON


In [147]:
! python3 main.py javascript_code1.txt

JAVASCRIPT


In [148]:
! python3 main.py javascript_code2.txt

JAVASCRIPT


In [149]:
! python3 main.py javascript_code3.txt

JAVASCRIPT


In [150]:
! python3 main.py c++_code1.txt

CPLUSPLUS


In [151]:
! python3 main.py c++_code2.txt

CPLUSPLUS


In [152]:
! python3 main.py c++_code3.txt

CPLUSPLUS
